### Import library
Nous allons importer les bibliothèque numpy et pandas qui sont necessaire pour le traitement.

In [17]:
import pandas as pd
import numpy as np

### Import Data
Nous allons importer les données depuis le répertoire RAwData pour enfin les traiter et transformer

In [204]:
k="87"  #numéro à incrémenter
fullname="milieu-etgauchederrierep27__02-11-2020_17h_27m_30s"
name="milieu-etgauchederrierep27" #fullname sans la data
power="27"

In [205]:
n=['TimesTamp', 'ECP', 'Antenna', 'RSSI','Channel','Address']
p='../1-RawData/datasetfinale/cible_parasites/'+power+'/'+fullname+'.csv'
d=','
def importData(path, delimit,cols):
    return pd.read_csv(path, sep=delimit,names=cols)

data=importData(p,d,n)
data = data.drop([0],axis=0) #Suppression de la 1ere ligne
data = data.drop([1],axis=0) #Suppression de la 1ere ligne
data = data.drop([2],axis=0) #Suppression de la 1ere 
data = data.drop(['Channel'],axis=1) #Suppression de la 1ere 
data = data.drop(['Address'],axis=1) #Suppression de la 1ere 
data['FP']=0
data.head(5)

,TimesTamp,ECP,Antenna,RSSI,FP
3,1581429688554361,E28068900000000018345804,1,-62.5,0
4,1581429688555691,E28068900000000018345317,1,-56,0
5,1581429688576800,90134BC1F1CE3781AEA1001A,1,-62,0
6,1581429688592010,E28068900000000018345317,1,-56,0
7,1581429688593466,E28068900000000018345804,1,-63.5,0


### TYPAGE DES CHAMPS
Nos allons typer les différentes colonnes du dataSet avec les types primaires de python

In [206]:
def typage(data):
    data['ECP']=data['ECP'].astype(str)
    data['TimesTamp']=data['TimesTamp'].astype('int64')
    data['RSSI']=data['RSSI'].astype('float64')
    data['Antenna']=data['Antenna'].astype('int64')
    data['FP']=data['FP'].astype('int64')
    return data

data=typage(data)
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 559 entries, 3 to 561
Data columns (total 5 columns):
TimesTamp    559 non-null int64
ECP          559 non-null object
Antenna      559 non-null int64
RSSI         559 non-null float64
FP           559 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 26.2+ KB


(559, 5)

In [207]:
'''
3005FB63AC1F3681EC880468, 1532AC20AC1F3681EC880468, 0000FB63AC1F3681EC880468, 1007FB63AC1F3681EC880468
'''
def deleteRowByECP(datas, ecp):
    print('firstSize', datas.size)
    datas=datas[datas.ECP!=ecp]
    print('secondSize', datas.size)
    return datas

data = deleteRowByECP(data,'3005FB63AC1F3681EC880468')
data = deleteRowByECP(data,'1532AC20AC1F3681EC880468')
data = deleteRowByECP(data,'0000FB63AC1F3681EC880468')
data = deleteRowByECP(data,'1007FB63AC1F3681EC880468')

firstSize 2795
secondSize 2525
firstSize 2525
secondSize 2505
firstSize 2505
secondSize 2305
firstSize 2305
secondSize 2125


In [208]:
parasites = ['3025AB20CA1F3681CF101116','3025AB20CA1F3681CF10111B','90134BC1F1CE3781AEA1000A',
'90134BC1F1CE3781AEA10009','E2806890000000001834545B','E28068900000000018345DFC',
'E28068900000000018345317','90134BC1F1CE3781AEA10012','9014FB63AC1F3681EC880901',
'90134BC1F1CE3781AEA10023','E2806890000000001834545C', '90134BC1F1CE3781AEA10021',
'90144BC1F1CE3681EA880912','90134BC1F1CE3781AEA10025','3025AB20CA1F3681CF101119',
'E28068900000000018344E55','E28068900000000018344F88', 'E28068900000000018344F9A',
'E28068900000000018344F87','E28068900000000018344AC4','E2806890000000001834580B',
'90134BC1F1CE3781AEA10002','3025AB20CA1F3681CF10111A','E28068900000000018345944',
'E28068900000000018345804','E28068900000000018344E66','E28068900000000018345329',
'E28068900000000018345CD5','E28068900000000018344F93','E2806890000000001834580A',
'4001FBCAAC1F3690ED880512','90134BC1F1CE3781AEA1001A', '300000000000000000017868', 
'30000000000000000001786C', '30000000000000000001786E', '300000000000000000017866', 
'30000000000000000001786D', '30000000000000000001786A', '300000000000000000017865', 
'300000000000000000017863', '3000000000000000000178BE', 'E28068900000000018345CCE',
'3025AB20CA1F3681CF101118','3025AB20CA1F3681CF10111D','E28068900000000018345455',
'90134BC1F1CE3781AEA1000F','E28068900000000018345CD4']
print(len(parasites))
parasites=list(set(parasites))
print(len(parasites))
data.shape

47
47


(425, 5)

### Set False Positive 
Nous allons mettre les faux positives à 1 dans la liste

In [209]:
def objectExist (objecttag,listofparasites): 
    for dp in listofparasites:
        if(objecttag==dp):
            return True
    return False
def setFalsePositive(fp_list, dataSet):
    lp=[]
    for dp in dataSet.values:
        if objectExist(dp[1],fp_list)==True:
            dp[4]=1
            lp.append(dp) 
        else :
            lp.append(dp)    
    l=lp
    
    df=pd.DataFrame(l, columns=['TimesTamp', 'ECP', 'Antenna', 'RSSI','FP'])
    return df

data= setFalsePositive(parasites,data)

### Rename ECP
Nous allons rajouter un suffix au nom des ECP pour pouvoir eviter les conflits quand il y aura un merge

In [210]:
data.ECP = data.ECP.values+k
data.head(5)

,TimesTamp,ECP,Antenna,RSSI,FP
0,1581429688554361,E2806890000000001834580487,1,-62.5,1
1,1581429688555691,E2806890000000001834531787,1,-56.0,1
2,1581429688576800,90134BC1F1CE3781AEA1001A87,1,-62.0,1
3,1581429688592010,E2806890000000001834531787,1,-56.0,1
4,1581429688593466,E2806890000000001834580487,1,-63.5,1


### Export Data
Nous allons sauvegarder les donées dans un fichier .csv

In [211]:
import time
def exportData(data, path):
    data.to_csv(path, index = None, header=False)

In [212]:
path='../1-RawData/datasetfinale/cible_parasites/Data_transformed_'+power+'/'+k+'-'+name+'.csv'
exportData(data, path)